# PXD030782
## Peroxisome-derived hydrogen peroxide can modulate the sulfenylation profiles of key redox signaling proteins in Flp-In T-REx 293 cells

[Pride link](https://www.ebi.ac.uk/pride/archive/projects/PXD030782)

[Article link](https://pubmed.ncbi.nlm.nih.gov/35557958/)

In [1]:
#Import libraries
os.chdir("/home/robbe/ionbot/")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass
import OxiAnalysis as OA
import importlib
importlib.reload(OA)

<module 'OxiAnalysis' from '/home/robbe/ionbot/OxiAnalysis.py'>

In [3]:
first = pd.read_csv('/home/robbe/ionbot/full_projects/PXD030782_first.csv')
del first["Unnamed: 0"]
del first["spectrum_title"]
del first["scan"]
first['modifications'] = first['modifications'].fillna("None")
first['Oxidant treatment'] = first['Oxidant treatment'].fillna("None")
first['Secondary treatment'] = first['Secondary treatment'].fillna("None")
first['Length of Secondary treatment (mins)'] = first['Length of Secondary treatment (mins)'].fillna(0)
first['Length of Secondary treatment (mins)'] = first['Length of Secondary treatment (mins)'].replace('0', 0)
first['Length of oxidant treatment (mins)'] = first['Length of oxidant treatment (mins)'].replace('0', 0)

In [4]:
print("The amount of PSMs in this project dataset is: ", first.shape[0])
print("The amount of raw files that make up the master dataset is: ", len(first["spectrum_file"].unique()))

The amount of PSMs in this project dataset is:  65490
The amount of raw files that make up the master dataset is:  12


## Overview of the treatments in this project

In [5]:
first.groupby(["Oxidant treatment", "Oxidant dose (µM)", "Length of oxidant treatment (mins)", "Secondary treatment", "Secondary treatment dose (µM)", 'Length of Secondary treatment (mins)'])["ionbot_match_id"].count().to_frame().reset_index()

,Oxidant treatment,Oxidant dose (µM),Length of oxidant treatment (mins),Secondary treatment,Secondary treatment dose (µM),Length of Secondary treatment (mins),ionbot_match_id
0,H2O2,10,10,c-YAP,0,0,6509
1,H2O2,30,10,c-YAP,0,0,7071
2,H2O2,100,10,c-YAP,0,0,7500
3,H2O2,300,10,c-YAP,0,0,7498
4,H2O2,1000,10,c-YAP,0,0,11183
5,H2O2,1000,10,mt-YAP,0,0,4079
6,H2O2,1000,10,po-YAP,0,0,3054
7,None,0,0,c-YAP,0,0,10299
8,None,0,0,mt-YAP,0,0,4314
9,None,0,0,po-YAP,0,0,3983


### Conditions that will be compared

| N | Treatment | Treatment 2 | Treatment 3|
| - | ----------- | ----------- | -------- |
| 1 | None c-YAP | None mt-YAP | None po-YAP |
| 2 | None c-YAP | H2O2 c-YAP | -- |
| 3 | None mt-YAP | H2O2 mt-YAP | -- |
| 4 | None po-YAP | H2O2 po-YAP | -- |

And then a concentration-dependend analysis with condition 0-4 en 7

In [7]:
H2O2c10 = first.loc[lambda first: (first["Oxidant dose (µM)"] == 10)]
H2O2c30 = first.loc[lambda first: (first["Oxidant dose (µM)"] == 30)]
H2O2c100 = first.loc[lambda first: (first["Oxidant dose (µM)"] == 100)]
H2O2c300 = first.loc[lambda first: (first["Oxidant dose (µM)"] == 300)]
H2O2c1000 = first.loc[lambda first: (first["Oxidant dose (µM)"] == 1000) &  (first["Secondary treatment"] == "c-YAP")]
H2O2mt1000 = first.loc[lambda first: (first["Oxidant treatment"] == "H2O2") & (first["Secondary treatment"] == "mt-YAP")]
H2O2po1000 = first.loc[lambda first: (first["Oxidant treatment"] == "H2O2") & (first["Secondary treatment"] == "po-YAP")]
Nonec = first.loc[lambda first: (first["Oxidant treatment"] == "None") & (first["Secondary treatment"] == "c-YAP")]
Nonemt = first.loc[lambda first: (first["Oxidant treatment"] == "None") & (first["Secondary treatment"] == "mt-YAP")]
Nonepo = first.loc[lambda first: (first["Oxidant treatment"] == "None") & (first["Secondary treatment"] == "po-YAP")]

## Comparison between the treatments
### 1) No treatment, with proteins isolated from cytosol, mitochondria and peroxisomes

In [12]:
print("There are this many replicates of None-c treatment: {}".format(Nonec["spectrum_file"].nunique()))
print("There are this many replicates of None-mt treatment: {}".format(Nonemt["spectrum_file"].nunique()))
print("There are this many replicates of None-po treatment: {}".format(Nonepo["spectrum_file"].nunique()))

There are this many replicates of None-c treatment: 2
There are this many replicates of None-mt treatment: 1
There are this many replicates of None-po treatment: 1


Check this??